In [74]:
''' Run MIP for indel inference '''

# libraries
import gurobipy as gp
from gurobipy import abs_,quicksum
from gurobipy import GRB
import time
import json
from collections import defaultdict
import numpy as np
import torch
from torch.utils.data import Dataset
import pickle

In [85]:
m = gp.Model("test")

a = '10010101'
b = '00101100'
len_a = len(a)

#temp_diff   = m.addVars(len_a,vtype=GRB.INTEGER,lb=-1,ub=1,name="diff")
diff      = m.addVars(len_a,vtype=GRB.BINARY,name="abs_diff")
pen =  m.addVars(len_a,vtype=GRB.BINARY,name="abs_diff")

diff_dict = {}
for i in range(0,len_a):
    
    m.addConstr( diff[i] <= int(a[i]) + int(b[i]) )
    m.addConstr( diff[i] >= int(a[i]) - int(b[i]) )
    m.addConstr( diff[i] >= int(b[i]) - int(a[i]) ) 
    m.addConstr( diff[i] <= 2 - int(b[i]) - int(a[i]) )
                
    if i == 0: # penalty for first position is simple
        m.addConstr(pen[i] == diff[i])
    else:
        m.addConstr((pen[i]==0)>>(diff[i] - diff[i-1] <= 0.0))
        m.addConstr((pen[i]==1)>>(diff[i] - diff[i-1] >= 1.0))
    
    
m.update()
m.optimize()
m.write('test.lp')
diff_all = []
pen_all = []
for i in range(0,len_a):
    diff_all.append(diff[i].X)
    pen_all.append(pen[i].X)
print("diff_all",diff_all)
print("pen_all",pen_all)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 33 rows, 16 columns and 34 nonzeros
Model fingerprint: 0x378a187a
Model has 14 general constraints
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 33 rows and 16 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
diff_all [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0]
pen_all [1.0, 0.0, 1.0, 0.0, 0.0, 0.